<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning_(FL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install syft

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy

# Create a virtual workers environment
hook = sy.TorchHook(torch)
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Distribute the model to the workers
model = SimpleNN()
model = model.send(alice)

# Training loop for federated learning
def train_federated(worker, data_loader, model, criterion, optimizer):
    model.train()
    model = model.send(worker)
    for data, target in data_loader:
        data, target = data.send(worker), target.send(worker)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    model = model.get()

# Example usage with synthetic data
# (Replace with actual data loaders for alice and bob)
data_loader_alice = [(torch.rand(64, 1, 28, 28), torch.randint(0, 10, (64,)))]
data_loader_bob = [(torch.rand(64, 1, 28, 28), torch.randint(0, 10, (64,)))]

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1, 5):
    train_federated(alice, data_loader_alice, model, criterion, optimizer)
    train_federated(bob, data_loader_bob, model, criterion, optimizer)
    print(f"Epoch {epoch} completed")